
# Evaluating Complex Functions Using Real Arithmetic

This notebook presents a method to evaluate complex functions using only real arithmetic. 
This is especially useful in environments where complex mathematics libraries are unavailable.

We'll follow the approach below:
1. Represent complex functions as arrays of operations.
2. Use real arithmetic to evaluate these operations.

Let's dive in!


In [1]:

from math import sin, sinh, cos, cosh, atan2


In [2]:

def complex_mul(z1, z2):
    """Multiply two complex numbers represented as tuples."""
    a1, b1 = z1
    a2, b2 = z2
    return (a1 * a2 - b1 * b2, a1 * b2 + a2 * b1)

def complex_cos(z):
    """Cosine of a complex number represented as a tuple."""
    a, b = z
    return (cos(a) * cosh(b), -sin(a) * sinh(b))

def complex_pow(z, n):
    """Raise a complex number represented as a tuple to a power."""
    a, b = z
    magnitude = (a**2 + b**2) ** (n/2)
    angle = atan2(b, a)
    return (magnitude * cos(n * angle), magnitude * sin(n * angle))

def evaluate_expression_real(expr, z_value_tuple):
    """Evaluates a complex function using only real arithmetic."""
    
    if isinstance(expr, (int, float)):
        return (expr, 0)
    
    if isinstance(expr, tuple) and len(expr) == 2 and all(isinstance(i, (int, float)) for i in expr):
        return expr
    
    operation = expr[0]
    
    if operation == "mul":
        return complex_mul(evaluate_expression_real(expr[1], z_value_tuple), evaluate_expression_real(expr[2], z_value_tuple))
    elif operation == "pow":
        return complex_pow(evaluate_expression_real(expr[1], z_value_tuple), expr[2])
    elif operation == "cos":
        return complex_cos(evaluate_expression_real(expr[1], z_value_tuple))
    elif operation == "sub":
        left = evaluate_expression_real(expr[1], z_value_tuple)
        right = evaluate_expression_real(expr[2], z_value_tuple)
        return (left[0] - right[0], left[1] - right[1])
    elif operation == "z":
        return z_value_tuple
    else:
        raise ValueError(f"Unsupported operation: {operation}")


In [3]:

import ast

def generate_expression(node):
    """Generates an array of operations for a given AST node."""
    
    if isinstance(node, ast.BinOp):
        if isinstance(node.op, ast.Mult):
            return ("mul", generate_expression(node.left), generate_expression(node.right))
        elif isinstance(node.op, ast.Sub):
            return ("sub", generate_expression(node.left), generate_expression(node.right))
        elif isinstance(node.op, ast.Pow):
            return ("pow", generate_expression(node.left), generate_expression(node.right))
        else:
            raise ValueError(f"Unsupported binary operation: {type(node.op)}")
    
    elif isinstance(node, ast.UnaryOp):
        if isinstance(node.op, ast.USub):
            return ("neg", generate_expression(node.operand))
        else:
            raise ValueError(f"Unsupported unary operation: {type(node.op)}")
    
    elif isinstance(node, ast.Call):
        if isinstance(node.func, ast.Name) and node.func.id == "cos":
            return ("cos", generate_expression(node.args[0]))
        else:
            raise ValueError(f"Unsupported function: {node.func.id}")
    
    elif isinstance(node, ast.Name):
        if node.id == "z":
            return "z"
        else:
            raise ValueError(f"Unsupported variable: {node.id}")
    
    elif isinstance(node, ast.Num):
        return node.n
    
    else:
        raise ValueError(f"Unsupported node type: {type(node)}")

def expression_from_string(s):
    """Generates the array of operations from a string representation of a function."""
    
    parsed_expr = ast.parse(s, mode='eval').body
    return generate_expression(parsed_expr)

# Generate the representation for the function "z^3 * cos(z) - 1"
expr_str = "z*z*z*cos(z)-1"
generated_expr = expression_from_string(expr_str)
generated_expr


('sub', ('mul', ('mul', ('mul', 'z', 'z'), 'z'), ('cos', 'z')), 1)

In [4]:

# Test the evaluator with the generated expression and z = (1, 1)
test_value_tuple = (1, 1)
print(generated_expr)
evaluate_expression_real(generated_expr, test_value_tuple)


(-0.6896646387365681, 3.6452554617880284)